In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import taichi as ti
import numpy as np
import math
from engine.mpm_solver import MPMSolver
import argparse
import os






ti.init(arch=ti.cuda)  # Try to run on GPU

gui = ti.GUI("Taichi Elements", res=512, background_color=0x112F41)

mpm = MPMSolver(res=(256, 256), E_scale=0.001)
mpm2 = MPMSolver(res=(256,256), E_scale=0.5)
mpm.set_gravity([0, -9.81])

E1 = 1e6
E2 = 1e3

mpm.add_spikes(
            sides=4,
            center=[0.15,0.31],
            velocity=[0.1,0],
            radius=0.1,
            width = 0.02,
            color=0xFFAAAA,
            sample_density=1,
            material=mpm.material_elastic,
            E=E2,
            nu=0.2)
mpm.add_cube(
    lower_corner=[0.,0],
    cube_size=[0.1,0.1],
    velocity=[0, 0],
    sample_density=1,
    material=mpm.material_elastic,
    E=E2,
    nu=0.2
)
mpm.add_cube(
    lower_corner=[0.9,0],
    cube_size=[0.1,0.1],
    velocity=[0, 0],
    sample_density=1,
    material=mpm.material_elastic,
    E=E2,
    nu=0.2
)
mpm.add_cube(
    lower_corner=[0.,0.1],
    cube_size=[1,0.1],
    velocity=[0, 0],
    sample_density=1,
    material=mpm.material_elastic,
    E=E1,
    nu=0.2
)
for frame in range(1):
    mpm.step(8e-3, print_stat=True)
    particles = mpm.particle_info()
    print(mpm.fan_center, mpm.t, mpm.omega)
    # gui.circles(particles['position'], radius=1.5, color=particles['color'])
    # gui.show(f'{args.out_dir}/{frame:06d}.png' if write_to_disk else None)

[Taichi] Starting on arch=cuda
needed substeps: 1025
CFL: 0.004044274961890244
num particles=8365
  frame time 7.023 s
  substep time 6.851 ms
[0.15074718 0.3096966 ] 0.008000000000000018 0.0


In [16]:

mpm.n_particles[None]

8365

In [20]:
mpm.mu_0_list[8000]

416666.65625

In [6]:
import taichi as ti

ti.init(arch=ti.cuda)

# Create a 1D vector field with shape (1,)
test = ti.Vector.field(1, dtype=ti.f32, shape=1)

@ti.kernel
def assign_value():
    test[0] = [1.0]

assign_value()

# Access the value
print(test[0])

[Taichi] Starting on arch=cuda
[1.]
